<a href="https://colab.research.google.com/github/KellyKimGit/Daddypeak_NLP/blob/main/%ED%85%8D%EC%8A%A4%ED%8A%B8%EC%A0%84%EC%B2%98%EB%A6%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import io
import re

In [2]:
# 텍스트 내용(csv 파일) 불러오기
#data = pd.read_csv("/content/drive/MyDrive/Daddypeak/의좋은_형제.csv")
#data.head()
# 텍스트 로드
file_path = '/content/drive/MyDrive/fairy_tale_book.txt'
with open(file_path, 'r', encoding='utf-8') as file:
    lines = file.readlines()

In [3]:
# konlpy 설치
! pip install konlpy
from konlpy.tag import Hannanum, Kkma, Komoran, Mecab, Okt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 48.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.3/465.3 kB 26.0 MB/s eta 0:00:00


In [4]:
# csv 파일 지정
#text = pd.read_csv("/content/drive/MyDrive/Daddypeak/의좋은_형제.csv").to_string(columns=['Unnamed: 0'])

# Create a DataFrame from the loaded text
data = pd.DataFrame({'Unnamed: 0': lines})

# Extract 'Unnamed: 0' column as text
text = data['Unnamed: 0'].to_string(index=False)

In [7]:
!pip install kss

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.3/131.3 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.4/26.4 MB 46.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for kss: filename=kss-4.5.4-py3-none-any.whl size=54467 sha256=978eccc047eb05340d4f2e52e594b063735e251ff25df159602c288001e83528
  Stored in directory: /root/.cache/pip/wheels/61/7b/ba/e620ef5d96a61cdd83bdee4c2bb4aec8a74de5d72fcbb00e80
  Created wheel for pecab: filename=pecab-1.0.8-py3-none-any.whl size=26646666 sha256=1126e6bf273bd4b6e372fbeca899d47ae0813f49bbc4aabbdf6971c21b3b8edc
  Stored in directory: /root/.cache/pip/wheels/5c/6f/b4/ab61b8863d7d8b1409def8ae31adcaa089fa91b8d022ec309d
Successfully built kss pecab


In [8]:
import kss

sentence_tokenized_text = []
for line in lines:
    line = line.strip()
    for sent in kss.split_sentences(line):
        sentence_tokenized_text.append(sent.strip())

For your information, Kss also supports mecab backend.
We recommend you to install mecab or konlpy.tag.Mecab for faster execution of Kss.
Please refer to following web sites for details:
- mecab: https://github.com/hyunwoongko/python-mecab-kor
- konlpy.tag.Mecab: https://konlpy.org/en/latest/api/konlpy.tag/#mecab-class



이제 `sentence_tokenized_text`에 문장 단위로 분리된 corpus가 저장되었습니다.

In [9]:
punct = "/-'?!.,#$%\'()*+-/:;<=>@[\\]^_`{|}~" + '""“”’' + '∞θ÷α•à−β∅³π‘₹´°£€\×™√²—–&'

In [10]:
punct_mapping = {"‘": "'", "₹": "e", "´": "'", "°": "", "€": "e", "™": "tm", "√": " sqrt ", "×": "x", "²": "2", "—": "-", "–": "-", "’": "'", "_": "-", "`": "'", '“': '"', '”': '"', '“': '"', "£": "e", '∞': 'infinity', 'θ': 'theta', '÷': '/', 'α': 'alpha', '•': '.', 'à': 'a', '−': '-', 'β': 'beta', '∅': '', '³': '3', 'π': 'pi', }

In [11]:
def clean_punc(text, punct, mapping):
    for p in mapping:
        text = text.replace(p, mapping[p])

    for p in punct:
        text = text.replace(p, f' {p} ')

    specials = {'\u200b': ' ', '…': ' ... ', '\ufeff': '', 'करना': '', 'है': ''}
    for s in specials:
        text = text.replace(s, specials[s])

    return text.strip()

In [12]:
cleaned_corpus = []
for sent in sentence_tokenized_text:
    cleaned_corpus.append(clean_punc(sent, punct, punct_mapping))

In [13]:
for sentence in cleaned_corpus:
    print(sentence)

농사가 그리 잘 되지는 못했으나 ,  가을이 되어 만곡이 익으매 그래도 농가
에서들은 기쁜 마음으로 벼를 베느라고 바빴습니다 .
성칠이도 동네 사람들의 조력을 얻어 가지고 자기 논의 벼를 베어서 ,  논두
덕에 널어 두었습니다 .
며칠간 햇볕에 말려서 거둬들일 작정이었습니다 .
"  나는 딸리는 식구가 없으니까 염려 없지마는 ,  형님은 식구가 많아서 ,  금
년같이 풍년들지 못한 해에는 지내시기가 곤란하실 터인데 ...  ...  .   "
하루는 이런 생각이 나서 ,
"  오냐 ,  그냥 갖다 드리면 받지 않으실 것이니까 ,  밤중에 넌지시 내 논의
벼를 옮겨다가 ,  형님 논두렁에 더 놓아 드려야겠다 .   "
하고 ,  그 날 밤에 형님도 동리 사람도 다 잠들기를 기다려서 넌지시 지게를
지고 나가서 자기 논두렁의 벼를 여러 차례 옮겨다가 형님네 논두렁에 보태
놓아 두고 돌아왔습니다 .
그러나 ,  이상한 일입니다 .
이튿날 나가 본즉 ,  훨씬 적어졌을 자기 논두렁
의 벼는 조금도 적어지지 않고 ,  전에 있던 그대로 있습니다 .
"  아니 ,  내가 분명히 어젯밤에 형님 논으로 여러 짐을 져다 두었는데 ,  이
것이 웬일일까 ?   "
하고 ,  형님 논으로 가 보니까 ,  거기도 전보다 별로 많아져 보이지 않습니
다 .
"  아니 ,  내가 꿈을 꾼 것인가 ?   "
하고 ,  그 날 온종일 이상히 여기다가 그 날 밤에 또 동리 사람이 잠들기를
기다려서 ,  넌지시 나가 자기 논의 벼를 여러 짐 져다가 형님의 논에 널어
두었습니다 .
그리고 ,  이튿날 나가 보니 참 이상합니다 .
그래도 자기 논에는 벼가 줄지
아니하였고 ,  형님 논에 늘지도 않았습니다 .
"  이건 참 귀신의 장난 같구나 !  대체 어찌된 까닭을 모르겠다 .   "
하고 ,  또 그 날 밤이 되기를 기다려서 ,  자기 논의 벼를 거둬서 짊어지고 형
님 논으로 갔습니다 .
마침 그 날은 날이 흐리어 별 하나도 없어서 몹시 캄캄하므로 ,  구렁에 빠
지지 않으려고 길바닥만 보면서 가

In [14]:
import re

def clean_text(texts):
    corpus = []
    for i in range(0, len(texts)):
        review = re.sub(r'[@%\\*=()/~#&\+á?\xc3\xa1\-\|\.\:\;\!\-\,\_\~\$\'\"]', '',str(texts[i])) #remove punctuation
        review = re.sub(r'\d+','', str(texts[i]))# remove number
        review = review.lower() #lower case
        review = re.sub(r'\s+', ' ', review) #remove extra space
        review = re.sub(r'<[^>]+>','',review) #remove Html tags
        review = re.sub(r'\s+', ' ', review) #remove spaces
        review = re.sub(r"^\s+", '', review) #remove space from start
        review = re.sub(r'\s+$', '', review) #remove space from the end
        corpus.append(review)
    return corpus

In [15]:
basic_preprocessed_corpus = clean_text(cleaned_corpus)

In [16]:
for i in range(len(basic_preprocessed_corpus)):
    print(basic_preprocessed_corpus[i])

농사가 그리 잘 되지는 못했으나 , 가을이 되어 만곡이 익으매 그래도 농가
에서들은 기쁜 마음으로 벼를 베느라고 바빴습니다 .
성칠이도 동네 사람들의 조력을 얻어 가지고 자기 논의 벼를 베어서 , 논두
덕에 널어 두었습니다 .
며칠간 햇볕에 말려서 거둬들일 작정이었습니다 .
" 나는 딸리는 식구가 없으니까 염려 없지마는 , 형님은 식구가 많아서 , 금
년같이 풍년들지 못한 해에는 지내시기가 곤란하실 터인데 ... ... . "
하루는 이런 생각이 나서 ,
" 오냐 , 그냥 갖다 드리면 받지 않으실 것이니까 , 밤중에 넌지시 내 논의
벼를 옮겨다가 , 형님 논두렁에 더 놓아 드려야겠다 . "
하고 , 그 날 밤에 형님도 동리 사람도 다 잠들기를 기다려서 넌지시 지게를
지고 나가서 자기 논두렁의 벼를 여러 차례 옮겨다가 형님네 논두렁에 보태
놓아 두고 돌아왔습니다 .
그러나 , 이상한 일입니다 .
이튿날 나가 본즉 , 훨씬 적어졌을 자기 논두렁
의 벼는 조금도 적어지지 않고 , 전에 있던 그대로 있습니다 .
" 아니 , 내가 분명히 어젯밤에 형님 논으로 여러 짐을 져다 두었는데 , 이
것이 웬일일까 ? "
하고 , 형님 논으로 가 보니까 , 거기도 전보다 별로 많아져 보이지 않습니
다 .
" 아니 , 내가 꿈을 꾼 것인가 ? "
하고 , 그 날 온종일 이상히 여기다가 그 날 밤에 또 동리 사람이 잠들기를
기다려서 , 넌지시 나가 자기 논의 벼를 여러 짐 져다가 형님의 논에 널어
두었습니다 .
그리고 , 이튿날 나가 보니 참 이상합니다 .
그래도 자기 논에는 벼가 줄지
아니하였고 , 형님 논에 늘지도 않았습니다 .
" 이건 참 귀신의 장난 같구나 ! 대체 어찌된 까닭을 모르겠다 . "
하고 , 또 그 날 밤이 되기를 기다려서 , 자기 논의 벼를 거둬서 짊어지고 형
님 논으로 갔습니다 .
마침 그 날은 날이 흐리어 별 하나도 없어서 몹시 캄캄하므로 , 구렁에 빠
지지 않으려고 길바닥만 보면서 가는데 , 저편에서 무언지 시꺼먼 것이 이쪽
으로 마주 오므로 깜짝 놀라서 

In [17]:
# okt를 사용한 형태소 분석
from konlpy.tag import Okt

okt = Okt()

print('OKT 형태소 분석 :',okt.morphs(text))
print('OKT 품사 태깅 :',okt.pos(text))
print('OKT 명사 추출 :',okt.nouns(text))

OKT 형태소 분석 : ['\\', 'n', '\n                                          ', '\\', 'n', '\n  ', '농사', '가', '그리', '잘', '되지는', '못', '했으나', ',', '가을', '이', '되어', '만', '곡', '이', '익으매', '그래도', '농가', '\\', 'n', '\n               ', '에서', '들은', '기쁜', '마음', '으로', '벼', '를', '베느라고', '바빴습니다', '.\\', 'n', '\n  ', '성', '칠이', '도', '동네', '사람', '들', '의', '조력', '을', '얻어', '가지', '고', '자기', '논의', '벼', '를', '베어서', ',', '논', '두', '\\', 'n', '\n    ', '덕', '에', '널어', '두었습니다', '.', '며칠', '간', '햇볕', '에', '말려서', '거둬들일', '작정', '이었습니다', '.\\', 'n', '\n  ', '“', '나', '는', '딸리는', '식구', '가', '없으니까', '염려', '없지마는', ',', '형님', '은', '식구', '가', '많아서', ',', '금', '\\', 'n', '\n        ', '년', '같이', '풍년', '들', '지', '못', '한', '해', '에는', '지', '내', '시기', '가', '곤란하실', '터', '인데', '…….', '”', '\\', 'n', '\n                            ', '하루', '는', '이런', '생각', '이', '나서', ',\\', 'n', '\n  ', '“', '오냐', ',', '그냥', '갖다', '드리면', '받지', '않으실', '것', '이니까', ',', '밤중', '에', '넌지시', '내', '논의', '\\', 'n', '\n             ', '벼', '를', '옮겨다가', ','

In [18]:
# 꼬꼬마를 사용한 형태소 분석
from konlpy.tag import Kkma

kkma = Kkma()

print('꼬꼬마 형태소 분석 :',kkma.morphs(text))
print('꼬꼬마 품사 태깅 :',kkma.pos(text))
print('꼬꼬마 명사 추출 :',kkma.nouns(text))

꼬꼬마 형태소 분석 : ['\\', 'n', '\\', 'n', '농사', '가', '그리', '잘', '되', '지', '는', '못하', '었', '으나', ',', '가을', '이', '되', '어', '만곡', '이', '익', '으매', '그리하', '여도', '농가', '\\', 'n', '에서', '듣', '은', '기쁘', 'ㄴ', '마음', '으로', '벼', '를', '베', '느라고', '바쁘', '았', '습니다', '.', '\\', 'n', '성', '치', 'ㄹ', '이도', '동네', '사람', '들', '의', '조력', '을', '얻', '어', '가지', '고', '자기', '논', '의', '벼', '를', '베', '어서', ',', '논', '두', '\\', 'n', '덕', '에', '널', '어', '두', '었', '습니다', '.', '며칠간', '햇볕', '에', '말리', '어서', '거둬들이', 'ㄹ', '작정', '이', '었', '습니다', '.', '\\', 'n', '“', '나', '는', '딸리', '는', '식구', '가', '없', '으니까', '염려', '없', '지마는', ',', '형님', '은', '식구', '가', '많', '아서', ',', '금', '\\', 'n', '년', '같이', '풍년', '들', '지', '못하', 'ㄴ', '해', '에', '는', '지내', '시기', '가', '곤란', '하', '시', 'ㄹ', '터', '이', 'ㄴ데', '……', '.', '”', '\\', 'n', '하루', '는', '이런', '생각', '이', '나', '아서', ',', '\\', 'n', '“', '오', '이', '냐', ',', '그냥', '갖', '다', '드리', '면', '받', '지', '않', '으시', 'ㄹ', '것', '이', '니까', ',', '밤중', '에', '넌지시', '내', '논의', '\\', 'n', '벼', '를', '옮기', '어', 